Посмотрим, как работает готовый Naive Bayes алгоритм

В библиотеке sklearn есть несколько байесовских классификаторов:

1. BernoulliNB работает с бинарными признаками
2. MultinomialNB работает с категориальными признаками
3. GaussianNB работает с непрерывными признаками
4. ComplementNB улучшенная версия MultinomialNB, работающая с сильно несбалансированными выборками

Работать будем с датасетом Gender_class.csv
В нем содержатся данные о предпочтениях в цветах, музыке, алкогольных и безалкогольных напитков для мужчин и женщин. Пол человека мы и будем пытаться угадать.

In [1]:
import pandas as pd

# LabelEncoder поможет нам подготовить данные для классификатора "из коробки" sklearn,
from sklearn.preprocessing import LabelEncoder

# naive bayes для категориальных признаков "из коробки"
from sklearn.naive_bayes import MultinomialNB

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics

df = pd.read_csv('Gender_class.csv')

In [2]:
# давайте посмотрим, что за датасет мы считали
df.sample(15)

,Favorite Color,Favorite Music Genre,Favorite Beverage,Favorite Soft Drink,Gender
29,Cool,Electronic,Doesn't drink,Fanta,F
45,Cool,Hip hop,Beer,Coca Cola/Pepsi,M
12,Warm,Pop,Wine,7UP/Sprite,F
49,Warm,Hip hop,Beer,Coca Cola/Pepsi,M
10,Cool,Pop,Other,7UP/Sprite,F
24,Cool,Folk/Traditional,Whiskey,7UP/Sprite,F
59,Cool,Pop,Whiskey,Other,M
36,Neutral,Rock,Doesn't drink,Coca Cola/Pepsi,M
61,Cool,Rock,Vodka,Coca Cola/Pepsi,M
56,Warm,Folk/Traditional,Other,Fanta,M


Проверьте самостоятельно, что мужчин и женщин в выборке поровну, и найдите общий размер выборки

In [3]:
len(df)

66

In [9]:
len(df[df['Gender']=='M'])

33

In [10]:
# если бы мы писали классификатор самостоятельно, можно было бы оставить категории в признаках,
# классификатору это абсолютно неважно.
# готовый naive bayes из sklearn на вход просит данные в числовом формате.
# LabelEncoder() каждой категории внутри признака ставит в соответствие число
le = LabelEncoder()
for i in range(5):
    df.iloc[:, i] = le.fit_transform(df.iloc[:, i])

In [11]:
# разделим датасет на признаки..
dfx = df.drop('Gender', axis = 1)
dfx.head()

,Favorite Color,Favorite Music Genre,Favorite Beverage,Favorite Soft Drink
0,0,6,3,0
1,1,2,3,1
2,2,6,5,1
3,2,1,4,2
4,0,6,3,1


In [14]:
# и целевую переменную
dfy = df[['Gender']]
dfy.head()

,Gender
0,0
1,0
2,0
3,0
4,0


In [15]:
# разделим данные на train и test
# зафиксируем random_state=180
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.3, random_state=180) 
X_train.head()

,Favorite Color,Favorite Music Genre,Favorite Beverage,Favorite Soft Drink
60,0,0,4,1
50,0,5,1,1
42,2,5,1,1
7,2,4,4,2
65,0,0,0,1


Параметры классификатора:

alpha - коэффициент регуляризации, нужен, чтобы классифицировать людей, чей любимый напиток не входит в обучающую выборку. По умолчанию равен 1

fit_prior - если False, то априорные вероятности классов считаются одинаковыми. Если True - априорные вероятности классов вычисляются по обучающей выборке

class_prior - позволяет явно задать априорные вероятности классов, если они нам откуда-то известны.

In [16]:
mnb = MultinomialNB(alpha=1, fit_prior=True, class_prior=None)

In [17]:
# обучаем модель
mnb.fit(X_train, y_train.values.ravel())

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [18]:
# делаем предсказания на тестовой выборке
y_pred = mnb.predict(X_test)
y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1])

In [19]:
# смотрим, где угадали и не угадали
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[7, 1],
       [7, 5]], dtype=int64)

Самостоятельно вычислите Accuracy, Чувствительность, Специфичность, Точность и вероятности ошибок 1 и 2 рода

Задача 8.7. 

Условие

Загрузите датасеты Admission_train.csv и Admission_test.csv.
Преобразуйте данные в столбце Chance of Admit в 0 и 1 по правилу: 0, если вероятность меньше 0.5, и 1, если больше либо равна 0.5. Удалите столбцы: Unnamed: 0 и Serial No.

In [20]:
df_train = pd.read_csv('Admission_train.csv')

df_train.drop(['Unnamed: 0'], axis=1, inplace=True)
df_train.drop(['Serial No.'], axis=1, inplace=True)

df_test = pd.read_csv('Admission_test.csv')

df_test.drop(['Unnamed: 0'], axis=1, inplace=True)
df_test.drop(['Serial No.'], axis=1, inplace=True)

df_train.columns=['GRE', 'TOEFL', 'University Rating', 'SOP', 'LOR', 'CGPA', 'Research', 'Chance']
df_test.columns=['GRE', 'TOEFL', 'University Rating', 'SOP', 'LOR', 'CGPA', 'Research', 'Chance']

def func(x):
    if x<0.5:
        return 0
    else:
        return 1

train=df_train.Chance.apply(func)
df_train.Chance=train

test=df_test.Chance.apply(func)
df_test.Chance=test   

X_train=df_train.drop('Chance', axis = 1)
y_train=df_train[['Chance']]
X_test=df_test.drop('Chance', axis = 1)
y_test=df_test[['Chance']]

Задание 8.7.1
 
Обучите НБК по всем признакам, используя GaussianNB. Вычислите на тестовой выборке Accuracy, чувстительность, специфичность, точность, вероятности ошибок 1 и 2 рода.

In [21]:
from sklearn.naive_bayes import GaussianNB

In [23]:
clf = GaussianNB()
clf.fit(X_train, y_train.values.ravel()) #делаем у_train одномерным массивом значений, так нужно

GaussianNB(priors=None, var_smoothing=1e-09)

In [29]:
y_pred = clf.predict(X_test)
y_pred

array([1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], dtype=int64)

In [30]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[ 8,  1],
       [10, 81]], dtype=int64)

In [32]:
TN = cnf_matrix[0,0] # True Negative
TP = cnf_matrix[1,1] # True Positive
FN = cnf_matrix[1,0] # False Negative
FP = cnf_matrix[0,1] # False Positive
    
Ac = clf.score(X_test, y_test)
Sens = TP/(TP+FN) 
Sp = TN/(TN+FP)
P = TP/(TP+FP)
typeI = FP/(FP+TN)
typeII = FN/(FN+TP)
    
print('Accuracy: ', Ac)
print('Sensitivity: ', Sens)
print('Specificity: ', Sp)
print('Pricision: ', P)
print('Type I error rate: ', typeI)
print('Type II error rate: ', typeII)

Accuracy:  0.89
Sensitivity:  0.8901098901098901
Specificity:  0.8888888888888888
Pricision:  0.9878048780487805
Type I error rate:  0.1111111111111111
Type II error rate:  0.10989010989010989
